In [1]:
import pytz
import os
import logging
import pandas as pd
from datetime import datetime
from typing import Tuple, List, Optional
from etherscan_functions import get_erc20_transfers, get_block_numbers_by_date


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

START_DATE = "2024-12-14 08:00"
END_DATE = "2024-12-16 08:00"
OUTPUT_FILE = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result'
api_key = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1" # my free API key, you can get one at https://etherscan.io/myapikey
base_url = "https://api.etherscan.io/v2/api" # We're using the v2 API 2024/12/12
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB" # Address of the contract we want to get the source code of
chain_id = 1  # Ethereum Mainnet

In [2]:
def fetch_erc20_transfers(address: str, start_block: int, end_block: int, offset: int = 100) -> pd.DataFrame:
    """
    Fetch ERC20 token transfer data and process it into a DataFrame.
    :param address: Contract address you decide to fetch the data from.
    :param start_block: Start block number.
    :param end_block: End block number.
    :param offset: Number of transactions to fetch per API call.
    """
    all_transfers = []
    page = 1

    logger.info(f"Fetching ERC20 transfers from block {start_block} to {end_block}...")

    while True:
        try:
            transfers = get_erc20_transfers(
                address=address, startblock=start_block, endblock=end_block, page=page, offset=offset
            )

            if transfers is None:
                logger.warning("API returned None. Exiting...")
                break
            if isinstance(transfers, list) and len(transfers) == 0:
                logger.info("Empty list received. Exiting...")
                break
            if isinstance(transfers, pd.DataFrame) and transfers.empty:
                logger.info("Empty DataFrame received. Exiting...")
                break

            if isinstance(transfers, list):
                transfers_df = pd.DataFrame(transfers)
            elif isinstance(transfers, pd.DataFrame):
                transfers_df = transfers
            else:
                logger.warning(f"Unexpected data format: {type(transfers)}. Exiting...")
                break

            if transfers_df.empty:
                logger.info("Empty DataFrame received. Exiting...")
                break

            all_transfers.extend(transfers_df.to_dict(orient='records'))
            logger.info(f"Page {page}: Retrieved {len(transfers_df)} transactions.")
            page += 1

        except Exception as e:
            logger.warning(f"API request failed on page {page}: {e}")
            break

    final_df = pd.DataFrame(all_transfers)
    logger.info(f"Final dataset contains {len(final_df)} unique transactions.")
    return final_df


In [3]:
def process_and_save_transfers(transfers_df: pd.DataFrame, output_file: str) -> None:
    """
    Process ERC20 transfers DataFrame and save to a CSV file.
    """
    if not transfers_df.empty:
        transfers_df['dateTime'] = pd.to_datetime(
            pd.to_numeric(transfers_df['timeStamp'], errors='coerce'), unit='s', utc=True
        ).dt.strftime('%Y-%m-%d %H:%M:%S')
        cols = ['dateTime'] + [col for col in transfers_df.columns if col != 'dateTime']
        transfers_df = transfers_df[cols]
        output_file = os.path.join(output_file, 'erc20_transfers.csv')

        transfers_df.to_csv(output_file, index=False, encoding='utf-8')
        logger.info(f"Data successfully saved to {output_file}")
    else:
        logger.warning("No valid transfers to save. DataFrame is empty.")

In [4]:
try:
    # get the block numbers
    start_block, end_block = get_block_numbers_by_date(start_date=START_DATE, end_date=END_DATE, include_all=False)
    print(start_block, end_block)
    # fetch the ERC20 transfers
    transfers_df = fetch_erc20_transfers(address=ADDRESS, start_block=start_block, end_block=end_block)
    
    process_and_save_transfers(transfers_df, OUTPUT_FILE)

except Exception as e:
    logger.error(f"An error occurred during the execution: {e}")


2024-12-17 12:03:40,064 - INFO - Fetching ERC20 transfers from block 21397030 to 21411357...


21397030 21411357


2024-12-17 12:03:41,250 - INFO - Page 1: Retrieved 94 transactions.
2024-12-17 12:03:42,199 - WARNING - API request failed on page 2: Request failed: No transactions found
2024-12-17 12:03:42,200 - INFO - Final dataset contains 94 unique transactions.
2024-12-17 12:03:42,200 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
